In [ ]:
import cv2
import numpy as np
import cv2.aruco as aruco
import winsound

# Cargar los parámetros de la cámara desde el archivo de calibración
calibration_data = np.load('calibration_params.npz')
camera_matrix = calibration_data['mtx']
dist_coeffs = calibration_data['dist']

# Configuración inicial de ArUco
aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_50)  # ArUco de 6x6
aruco_params = aruco.DetectorParameters_create()

# Captura de video desde la cámara (puede ser DroidCam o cualquier otra fuente)
cap = cv2.VideoCapture('http://192.168.3.26:4747/video')

# Inicializar sustractor de fondo
backSub = cv2.createBackgroundSubtractorMOG2()

# Factor de desplazamiento para ampliar el área alrededor del marcador (ajústalo según necesites)
expand_by = 100  # Puedes ajustar este valor

intrusion_detected = False
alarm_active = False

# Variable para controlar cada cuántos cuadros procesar
process_every_n_frames = 5
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Solo procesar 1 de cada `process_every_n_frames` cuadros
    if frame_count % process_every_n_frames != 0:
        continue

    # Corregir la distorsión de la imagen
    h, w = frame.shape[:2]
    new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, (w, h), 1, (w, h))
    undistorted_frame = cv2.undistort(frame, camera_matrix, dist_coeffs, None, new_camera_matrix)

    # Recortar el área útil
    x, y, w, h = roi
    undistorted_frame = undistorted_frame[y:y+h, x:x+w]

    # Detección de marcadores ArUco en la imagen corregida
    corners, ids, _ = aruco.detectMarkers(undistorted_frame, aruco_dict, parameters=aruco_params)
    
    if ids is not None:
        for i in range(len(ids)):
            c = corners[i][0]
            
            # Expande cada esquina del área prohibida alrededor del marcador
            top_left = [c[0][0] - expand_by, c[0][1] - expand_by]
            top_right = [c[1][0] + expand_by, c[1][1] - expand_by]
            bottom_right = [c[2][0] + expand_by, c[2][1] + expand_by]
            bottom_left = [c[3][0] - expand_by, c[3][1] + expand_by]
            
            # Crear el área prohibida ampliada alrededor del marcador en el suelo
            prohibited_area = np.array([
                top_left,
                top_right,
                bottom_right,
                bottom_left
            ], dtype=np.int32)
            
            # Dibujar el área prohibida en la imagen (color verde para mayor visibilidad)
            cv2.polylines(undistorted_frame, [prohibited_area], isClosed=True, color=(0, 255, 0), thickness=3)

            # Detección de movimiento e intrusión
            fg_mask = backSub.apply(undistorted_frame)
            contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            for cnt in contours:
                if cv2.contourArea(cnt) > 500:  # Filtrar pequeñas detecciones
                    x, y, w, h = cv2.boundingRect(cnt)
                    
                    # Verificar si el intruso está dentro del área prohibida
                    for point in [(x, y), (x+w, y), (x+w, y+h), (x, y+h)]:
                        if cv2.pointPolygonTest(prohibited_area, point, False) >= 0:
                            # Activar alarma visual
                            cv2.putText(undistorted_frame, 'INTRUSION DETECTED', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                            intrusion_detected = True
                            break

    # Si se detecta una intrusión, activar el sonido de alarma
    if intrusion_detected and not alarm_active:
        winsound.Beep(1000, 100)  # Emitir un pitido a 1000 Hz por 100 ms
        alarm_active = True
    elif not intrusion_detected and alarm_active:
        # Desactivar alarma (podrías añadir aquí alguna señal acústica para indicar el final)
        alarm_active = False

    # Mostrar el video en tiempo real con el área prohibida dibujada
    cv2.imshow('Intrusion Detection', undistorted_frame)
    
    # Restablecer la detección de intrusión para la próxima iteración
    intrusion_detected = False

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
